# Quantum Chemistry with Paddle Quantum's qchem
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

qchem, which builds on top of Paddle Quantum, is designed to be a toolkit for quantum chemistry research in quantum computing era. It provides high level APIs for researchers who are interested in leveraging their current quantum chemsitry calculation with quantum computing power. And it also allows experts to write customized code. qchem is currently under active development, feel free to join us by opening issues or pull requests.

## Calculate ground state energy of a molecule
qchem provides `run_chem` function to calculate the ground state energy and ground state wave function. For example, let's try to calculate the ground state energy of hydrogen molecule. First, we need to import qchem.

In [ ]:
from paddle_quantum import qchem

Then, we need to provide chemical information required by `run_chem` function, including geometry, charge, basis function etc.

In [ ]:
# define the geometry of hydrogen molecule, length unit use angstrom.
h2_geometry = [("H", [0.0, 0.0, 0.0]), ("H", [0.0, 0.0, 0.74])]
charge = 0
basis_set = "sto-3g"

Next, let's choose an ansatz of many-electron wave function for our ground state energy calculation. Currently, you can choose to use "hardware efficient" [<sup>1</sup>](#refer-1) or "hartree fock" [<sup>2</sup>](#refer-2) ansatz, more functionalities will be released in the future. Let's choose "hardware efficient" ansatz.

In [ ]:
# call run_chem function with "hardware efficient" ansatz.
h2_gs_en, h2_wf_model = qchem.run_chem(
    h2_geometry,
    "hardware efficient",
    basis_set, 
    charge
)

# additional information for optimizer can be passed using `optimizer_option` keyword argument.
h2_gs_en, h2_wf_model = qchem.run_chem(
    h2_geometry,
    "hardware efficient",
    basis_set, 
    charge,
    optimizer_option={"learning_rate": 0.6, "weight_decay": 0.9}
)

# additional information for ansatz can be passed using `ansatz_option` keyword argument, e.g.
# "hardware efficient" ansatz has a parameter "cir_depth", which can be used to specify the depth
# of quantum circuit.
h2_gs_en, h2_wf_model = qchem.run_chem(
    h2_geometry,
    "hardware efficient",
    basis_set, 
    charge,
    ansatz_option={"cir_depth": 5}
)

You can also specify `max_iters` and `a_tol` keyword arguments to control the maximum iteration cycles and convergence criteria of the optimization process. 

To try another ansatz, you just need to replace the ansatz argument to, e.g. "hartree fock", and run similar command.

To see the quantum circuit for hydrogen molecule's hardware efficient ansatz, you can run `print(h2_wf_model.circuit)`.

## Design your own ansatz
For those who want to try an ansatz that isn't currently included in qchem, we provide method to write your own ansatz. Writing your own ansatz is similar to defining an neural network in paddlepaddle, except that your ansatz should inherit from `Qmodel`.

In [ ]:
from paddle_quantum.circuit import UAnsatz
from paddle_quantum import qchem
from paddle_quantum.qchem.layers import CrossResonanceLayer, EulerRotationLayer


# Your own model should inherit from `Qmodel`.
## NOTE: THIS MODEL IS ONLY DEFINED FOR DEMONSTRATION PURPOSE! 
class MyAnsatz(qchem.QModel):
    def __init__(self, n_qubits):
        super().__init__(n_qubits)

        self.entangle = CrossResonanceLayer(self._n_qubit)
        self.rot = EulerRotationLayer(self._n_qubit)

    def forward(self, state):
        self._circuit = UAnsatz(self.n_qubit)

        out = self.entangle(state)
        self._circuit += self.entangle.circuit

        out = self.rot(out)
        self._circuit += self.rot.circuit

        out = self.entangle(out)
        self._circuit += self.entangle.circuit

        return out

# instantiate your model
my_cir = MyAnsatz(5)

You can then follow the optimization procedure [here](https://qml.baidu.com/tutorials/quantum-simulation/variational-quantum-eigensolver.html) and use any paddlepaddle optimizer to train the ansatz you have built.

In [ ]:
# use paddlepaddle's optimizer
import numpy as np 
import paddle

optimizer = paddle.optimizer.Adam(parameters=my_cir.parameters(), learning_rate=0.08)

# define the loss function
## NOTE: THIS LOSS FUNCTION IS ONLY DEFINED FOR DEMONSTRATION PURPOSE!
def loss_fn(state: paddle.Tensor) -> paddle.Tensor:
    return paddle.norm(state.real())

# start learning
s0 = np.zeros((2**5,), dtype=np.complex128)
s0[0] = 1.0+0.0j
s0 = paddle.to_tensor(s0)

for i in range(10):
    loss = loss_fn(my_cir(s0))
    print(f"At {i:>d}th step: loss={loss.item():>.5f}.")

    optimizer.clear_grad()
    loss.backward()
    optimizer.step()

---
## References

[1] Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." [Nature 549.7671 (2017): 242-246.](https://www.nature.com/articles/nature23879)

[2] Arute, Frank, et al. "Hartree-Fock on a superconducting qubit quantum computer." [Science 369.6507 (2020): 1084-1089.](https://www.science.org/doi/10.1126/science.abb9811)